# Imports

In [1]:
# import required modules 
import requests, json 

# Pulling weather data

Code from https://www.geeksforgeeks.org/python-find-current-weather-of-any-city-using-openweathermap-api/

In [58]:
def get_weather(city_name):
    """
    Takes a city name, and returns a dictionary of weather and location data. 
    """
    # Python program to find current  
    # weather details of any city 
    # using openweathermap api 

    # Enter your API key here 
    api_key = "9291be5e6daedb7e3a4a03f8857a8d5f"

    # base_url variable to store url 
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # complete_url variable to store 
    # complete url address 
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name 

    # get method of requests module 
    # return response object 
    response = requests.get(complete_url) 

    # json method of response object  
    # convert json format data into 
    # python format data 
    x = response.json() 
    
    # Now x contains list of nested dictionaries 
    # Check the value of "cod" key is equal to 
    # "404", means city is found otherwise, 
    # city is not found 
    if x["cod"] != "404": 
        output_dict = {}
        output_dict['temp'] = x['main']['temp']
        output_dict['feels_like'] = x['main']['feels_like']
        output_dict['humidity'] = x['main']['humidity']
        output_dict['clouds'] = x['clouds']['all']
        output_dict['wind_speed'] = x['wind']['speed']
        if 'rain' not in x.keys():
            output_dict['rain'] = 0
        else:
            output_dict['rain'] = x['rain']['1h']
        if 'snow' not in x.keys():
            output_dict['snow'] = 0
        else:
            output_dict['snow'] = x['snow']['1h']
        return output_dict 
    else: 
        print(" City Not Found ") 
        return None

In [59]:
atx = get_weather("Austin")
atx

{'temp': 291.47,
 'feels_like': 289.99,
 'humidity': 48,
 'clouds': 90,
 'wind_speed': 1.15,
 'rain': 0,
 'snow': 0}

# Returning weather assessment

A function for returning the predicted response of the user to the weather (negative or positive)

In [61]:
def assess_weather(weather_dict, user_preferences = None):
    """
    Takes as input a dictionary with the weather parameters of interest:
        - temp
        - feels_like
        - humidity
        - clouds 
        - wind_speed
        - rain
        - snow
    Current implementation looks for a dictionary with a 'main' key, coresponding to a subdictionary
        which contains a 'temp' entry 
    Also takes user_preferences which is 
        TODO: DECIDE HOW TO INPUT THE USER PREFERENCES
    if user_preferences = None, use default temp cutoff
    Returns -1 or 1, for negative or positive predicted user response to weather.
    """
    temp = weather_dict['feels_like'] # temperature in Kelvin
    if user_preferences == None:
        if temp > 294: # corresponds to 70 F 
            return 1
        else:
            return -1
    else:
        pass

In [62]:
assess_weather(atx, None)

-1

# Learning user preferences

Suppose that a user's opinion of the weather is based on the average temperature, temperature spread, precipitation, and cloud coverage. 
Further suppose that a user's weather preferences may be similar to the preferences in their region (for cold start). 

Our initialization procedure can therefore be 

In [ ]:
def learn_preferences()

# Recommendation on sad days

For this, we can have a few simple options
* Inspirational quote
* * Weather related quotes? 
* * Fun facts
* Recommendation for mitigating the negative effect of the weather directly (umbrella, sunscreen, etc) 
* Netflix recommendation

In [55]:
def bad_weather_rec():
    quotes = [
        "The greatest glory in living lies not in never falling, but in rising every time we fall.", 
        "The way to get started is to quit talking and begin doing.", 
        "Your time is limited, so don't waste it living someone else's life. Don't be trapped by dogma – which is living with the results of other people's thinking."
    ]
    return np.random.choice(quotes)

In [56]:
bad_weather_rec()

'The way to get started is to quit talking and begin doing.'

# Give result to user

In [ ]:
def get_user_preferences(user):
    """
    
    """

In [ ]:
def main(user, )